In [2]:
import tensorflow.keras
tensorflow.keras.backend.set_learning_phase(0)
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.framework import convert_to_constants
import tensorflow_addons as tfa
from tensorflow_addons.image import resampler
import os
import cv2
import matplotlib.pyplot as plt

tfa.image.resampler

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import numpy as np
model_dir = "./config/hfnet_v2"
output_saved_model_dir = "./config/hfnet_v2_trt" 
output_frozen_path = "./config/hf_frozen_208x400_v2.pb"
verify_images_path= "/ssd/drone_data_process/images"


img = cv2.imread("./db1.jpg")
img = cv2.resize(img, (400, 400))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = img[0:208, 0:400]
tf.test.is_gpu_available()

tfa.register.register_all()

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (400, 208))
        if img is not None:
            images.append(img)
    return images
images = load_images_from_folder(verify_images_path)
print(f"Load {len(images)} images")

1 Physical GPUs, 1 Logical GPUs
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Load 5735 images


In [ ]:
class Net(object):

    def __init__(self, model_path):
        self.model_filepath = model_path
        self.load_graph(model_filepath=self.model_filepath)


    def load_graph(self, model_filepath):
        print("Loading model...")


        with tf.io.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())

        with tf.compat.v1.Session() as sess:
            with tf.Graph().as_default() as graph:
                tf.import_graph_def(graph_def, name='')
                signature = tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
                    inputs={'image':graph.get_tensor_by_name('image:0')},
                    outputs={'global_descriptor': graph.get_tensor_by_name('global_descriptor:0'),
                        'keypoints': graph.get_tensor_by_name('keypoints:0'),
                        'local_descriptors': graph.get_tensor_by_name('local_descriptors:0')}
                )
                builder = tf.compat.v1.saved_model.Builder("./config/hfnet_v2/")
                builder.add_meta_graph_and_variables(
                    sess=sess,
                    tags=[tf.compat.v1.saved_model.tag_constants.SERVING],
                    signature_def_map={'serving_default': signature}
                )
                builder.save()


Net(output_frozen_path)

In [3]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(
    max_workspace_size_bytes=(1<<30))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(
    maximum_cached_engines=100)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=model_dir,
    conversion_params=conversion_params)
converter.convert()
# np.shuffle()
def my_input_fn():
    for i in range(100):
        img = images[i]
        _img = np.expand_dims(img, axis=2)
        _img = np.array([[_img]]).astype(np.float32)
        #print(_img.shape)
    yield _img
converter.build(input_fn=my_input_fn)
converter.save(output_saved_model_dir)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 0)
ERROR:tensorflow:Loaded TensorRT 7.1.0 but linked TensorFlow against TensorRT 7.1.3. TensorRT does not support forward compatibility. It is also required to use the same major version of TensorRT during compilation and runtime.
INFO:tensorflow:Loaded TensorRT 7.1.0 and linked TensorFlow against TensorRT 7.1.3. This is supported because TensorRT  minor/patch upgrades are backward compatible
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./config/hfnet_v2_trt/assets


In [2]:
import time
start_time = time.time()
saved_model_loaded = tf.saved_model.load(
    output_saved_model_dir, tags=[tag_constants.SERVING])
# saved_model_loaded = tf.saved_model.load(
#     output_saved_model_dir)

frozen_func = graph_func = saved_model_loaded.signatures[
   signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

#frozen_func = convert_to_constants.convert_variables_to_constants_v2(
#    graph_func)
print("--- Loading Graph %s seconds ---" % (time.time() - start_time))


--- Loading Graph 14.497892618179321 seconds ---


In [3]:
def run(img):
    _img = np.expand_dims(img, axis=2)
    _img = np.array([_img]).astype(np.float)
    _img = tf.convert_to_tensor(_img, dtype=tf.float32)
    #print(_img.shape)
    output = frozen_func(_img)
    return output

img = cv2.imread("./db1.jpg")
img = cv2.resize(img, (400, 400))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = img[0:208, 0:400]
print("First Try")
start_time = time.time()
ret = run(img)
print("--- First trial %s seconds ---" % (time.time() - start_time))

start_time = time.time()
def draw(img, ret):
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    kps = ret["keypoints"][0].numpy()
    for kp in kps:
        img = cv2.circle(img, (kp[0], kp[1]), 3, (255, 255, 255))
    plt.imshow(img)
    plt.show()

for i in range(0, len(images)):
    _time = time.time()
    ret = run(images[i])
    if (time.time() - _time > 0.01):
        print(f"Img {i} cost {(time.time() - _time)*1000}ms")
        draw(images[i], ret)
        
print(f"--- Try {len(images)} images, average {(time.time() - start_time)/len(images)*1000} ms ---")



First Try


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  Input to reshape is a tensor with 332800 values, but the requested shape has 7680
	 [[{{node PartitionedCall/pred/global_head/vlad/Reshape}}]]
	 [[PartitionedCall/pred/descriptor_sampling/mul/_22]]
  (1) Invalid argument:  Input to reshape is a tensor with 332800 values, but the requested shape has 7680
	 [[{{node PartitionedCall/pred/global_head/vlad/Reshape}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_signature_wrapper_1110]

Function call stack:
signature_wrapper -> signature_wrapper


In [4]:
img.shape

(208, 400)